In [13]:
cd /content/drive/MyDrive/GRU-leakgan

/content/drive/MyDrive/GRU-leakgan


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing packages


In [3]:
import numpy as np
import pickle
from functools import reduce

Text Preprocessing


In [4]:
def generate_vocab(filePath, vocabpath):
    """
        Read text from file
    """
    with open(filePath, 'r') as f:
        lines = f.readlines()
    f.close()
    corpus = []
    for l in lines:
        l = l.strip().split(' ') 
        if len(l) < 28:
            corpus.append(l)

    vocab = []
    for p in corpus:
        vocab.extend(p)
    vocab = list(set(vocab)) 
    for i in range(len(vocab)):
        if vocab[i] == "<R>":
            break
    del vocab[i]
    vocab.append("<R>")

    corpus_num = []
    for sentence in corpus:
      for word in sentence:
        if word in vocab:
          continue
        else:
          vocab.append(word)

    # In order to generate vocabulary file for tensorflow codebase please uncomment the following lines
    # vocab_tf = []
    # for i in range(len(vocab)):
    #   tup = (vocab[i], str(i)) # word, encoding pair
    #   vocab_tf.append(tup)

    f = open(vocabpath, "wb") 
    pickle.dump(vocab, f)

def generate_arrData(filepath, seqlen, vocabpath):
  vocab = pickle.load(open(vocabpath, 'rb'))
  arr = np.empty((0,seqlen), int)
  with open(filepath, 'r') as fin:
    corpus_num = []
    for line in fin:
      line = line.split()
      line = line[0:seqlen]
      for word in line:
        corpus_num.append(vocab.index(word)+1)
      arr = np.vstack((arr,np.asarray(corpus_num)))
      corpus_num.clear()
  return arr

def tensor_to_text(outputfilepath, arraypath, vocabpath):
  vocab = pickle.load(open(vocabpath, 'rb'))
  with open(outputfilepath, 'w') as fout:
    gen = np.load(arraypath)
    for i in range(gen.shape[0]):
      words = [int(x) for x in gen[i]]
      words = [vocab[x-1] for x in words]
      words = ' '.join(words) + '\n'
      fout.write(words)

In [5]:
dataset_text = "data/realtrain_cotra_text.txt"
vocab_file = "data/chars.pkl"
# generate_vocab(dataset_text, vocab_file)

In [6]:
vocab = pickle.load(open(vocab_file, 'rb'))
len(vocab)

4797

In [8]:
seqlen = 10
data_arr = generate_arrData(dataset_text, seqlen, vocab_file)
data_arr

array([[4731, 4737,  712, ..., 2543,  246, 2674],
       [2383,  258, 1698, ..., 3058, 2114, 4698],
       [2004, 1486, 3890, ..., 1487, 2029, 3067],
       ...,
       [4731, 2075, 1078, ..., 1029, 1441, 1796],
       [4731,  909,  712, ..., 2777, 2029,  787],
       [3138, 1815, 3877, ..., 3997,  712, 1625]])

In [9]:
np.save("data/corpus", data_arr)

In [10]:
data_train = data_arr[0:40000]
np.save("data/train_corpus", data_train)
data_test = data_arr[40000:45000]
np.save("data/test_corpus", data_test)

In [ ]:
!python2 main.py

#########################################################################
Start Pretraining Discriminator...
Epoch: 0/10  Pre-Discriminator
/usr/local/lib/python2.7/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
Pre-Discriminator loss: 0.55170
Epoch: 1/10  Pre-Discriminator
Pre-Discriminator loss: 0.09769
Epoch: 2/10  Pre-Discriminator
Pre-Discriminator loss: 0.03683
Epoch: 3/10  Pre-Discriminator
Pre-Discriminator loss: 0.02435
Epoch: 4/10  Pre-Discriminator
Pre-Discriminator loss: 0.01297
Epoch: 5/10  Pre-Discrimi

In [11]:
genfilepath = "data/generated_file.txt"
genarraypath = "data/gen_corpus.npy"
reffilepath = "data/reference_file.txt"
refarraypath = "data/corpus.npy"

tensor_to_text(genfilepath, genarraypath, vocab_file)
tensor_to_text(reffilepath, refarraypath, vocab_file)

In [14]:
!python3 eval_bleu.py

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
LeakGAN
2 -gram BLEU score:  0.297822503767112
LeakGAN
3 -gram BLEU score:  0.21527181776608878
LeakGAN
4 -gram BLEU score:  0.1260470864408072
LeakGAN
5 -gram BLEU score:  0.1323972738163015
